Guardiamo la seconda parte del SN coding assignment. 

Fa GPR: il fit è illustrato nel notebook. Ci sono *sei* hyperparameters! **FAI CROSS-VALIDATION!**

La GPR dà anche incertezza sul fit. Not bad. Conclusioni: la GPR non è sempre praticabile.

Fa *parametric fit*: recap solito. Fa' Bayesian Fit! Fa' anche Frequentist Fit (con che assumptions?)!

Stimo $\mu(z)$ -> uso astropy.cosmology per calcolare quell'integrale.

Fa un MCMC (5-10 min) -> setup, run tutto presente nella soluzione.

Corner plot: H0 e Omega_Matter assumendo k=0.

Fa nested sampling, per controllare se viene lo stesso risultato. Trasforma la prior, usa dynesty. Ok, sembra convergere. Poco denso at alto -lnX, forse devi aumentare accuracy nel fit // far andare il codice più a lungo, per migliorare il risultato.

Output: un altro corner plot, suspiciously similar a quel che ho fatto prima. 

Plot: multiple realizations from the model (curve rosse)

Molto bene, ho misurato parametri cosmologici da SN data.

*Is there any dark energy?*

FORZO Omega_Lambda = 0 -> Il mio corner plot è un istogramma, ma come fitto i dati? Anche senza Dark Energy, non stai fittando male!!! Infatti se confronti i modelli, hai un 2 (VERY SMALL!)

Chiaramente, il problema è che NON HAI I PUNTI AD ALTO z SU CUI FITTARE, e A BASSO REDSHIFT OGNI MODELLO "VEDE" LA STESSA COSA.

Generiamo dei punti a caso e vediamo se spunta la Dark Energy?

Genero **mock data** -> random draws from nested sampling, e altri da GPR. BEH, diciamo che il modo in cui fa i random draws è molto più raffinato di quello visto all'inizio. Il metodo in giallo è estremamente data-driven, quello in blu fa delle assunzioni sull'universo.

> Abbiamo visto il fitting in modo frequentista, il fitting in modo bayesiano, e ora abbiamo visto il fitting da un punto di vista puramente di machine learning. Direi che abbiamo finito.

Proposta: prova a fittare per la curvatura dell'universo..?

**Sei altamente incoraggiato a fare ulteriori esplorazioni sui dati, al di là di quel che è stato richiesto!**

# Classification I

**SUPERVISED ML**. Voglio classificare i dati: so la truth di pochi oggetti, *labelled* e voglio inferire quella di *unlabelled* object.

* **Generative Classification**: what is the most likely category for the object i'm trying to classify? -> alla fine, è Density Estimation; 

* **Discriminative Classification**: ti interessano i boundaries, tra le regioni di classificazione. Non ti interessa classificare i singoli oggetti. E' più facile e meno ambiziosa dell'altra qui sopra, e a volte è quel che ti basta.

##### Assessing Results

Cosa significa che una classificazione è buona o cattiva?

Supponiamo di fare un covid test: il risultato sarà uno tra true positive, false positive, true negative, false negative.

Ciascun risultato ha una probabilità. Ci sono un po' di definizioni nel notebook:

* completeness: probabilità di trovare qualcosa di vero
* contamination: probabilità di trovare qualcosa di falso
* efficiency = precision = 1-contamination

Supponiamo di cercare qualcosa di super raro: non mi importa avere i falsi positivi, tanto quel che voglio è avere QUELL'evento (es. una SNe estremamente luminosa)

Supponiamo invece di cercare qualcosa con altissima precisione: a questo punto, li voglio sottoterra

> A seconda di cosa cerco, faccio diverse classificazioni con diversi criteri.

### Comparing Classifier Performances

ROC Curve: vedi notebook. Da quel che ho capito, la sua forma sarà diversa a seconda di cosa vuoi massimizzare.

Guarda tutto il blocco di codice: a seconda del threshold, hai una diversa efficiency e completeness. Nota che le due cose sono parecchio collegate!

Plot successivo: voglio precision 90%, che precision e recall mi servono?

Ok, tutto questo era per True/False e simili. Posso avere anche più di due opzioni, ma non approfondiamo l'argomento.

# Generative Classification

Basically, boils down to bayes's theorem.

Generative classifier: tutto quel che devo fare è *modellare la conditional probability* -> regression / density estimation problem!

Come chiamo la discriminant function?

* **Discriminant Function**: la conditional probability era l'obiettivo principale della regression. Nel nostro caso, f(y|x) ha *discrete* classes. 

* **Bayes Classifier**: se voglio solo una prediction, chiamo così la discriminant function

* **Decision Boundary**: un set di valori per cui ogni classe è equally likely 

(?) (?) (?) (?) (?)

### Naive Bayes Classifier

HP: tutti gli attributi sono *indipendenti*

Esempio con due dimensioni: scrive p come il prodotto delle probabilità, e applica la bayes rule.

Massimizziamo $\hat{y}$: mi servono 

* la frequency of the class k
* la probability density di un oggetto di classe k che abbia gli attributi x (è un density estimation problem)

#### Gaussian Naive Bayes

Qui diventa tutto analitico, e puoi trovare la forma di $\hat{y}$ indicata. In sklearn è implementato con `GaussianNB`. Trovo il boundary illustrato in figura.

Caso più astrofisico: sample di stelle da SDSS. Cerchiamo di estrarre delle stelle molto rare, le "RL Lyrae Variables". Sono variable stars, ci fai cose molto interessanti, e sono in verde nel plot. 

Il codice è in figura. Light points: variable source; dark points: non-variable source. 

Ok, quale regione è most likely per contenere le mie variable stars? Chiaramente, il mio set non è completo ed è contaminato!

> Domanda: che succede se "tiri su" la boundary condition "a mano"? Che succede a completezza e contaminazione? <br> Che succede invece se aumenti la dimensione del training set..?

Droppiamo l'assunzione che i singoli colori siano indipedenti: che succede? 

### Linear & Quadratic Discriminant Analysis

DROPPO l'assunzione che sia tutto indipendente da tutto, ma devo fare delle approssimazioni: butto quel che è più di linear/quadratic nella varianza.

Linear boundary, trova linea. Quadratic boundary, trova una curva.

**È molto simile al risultato precedente!** Motivo? I filtri dei colori sono abbastanza separati da permettermi di poter effettivamente assumere l'indipendenza dei colori...

### GMM and Bayes Classification

Alla fine mi sono ridotto a fare density estimation. 

Gaussian Model: dovevo specificare il numero di gaussiane. Una gaussiana, non basta sicuramente; con cinque colori e cinque gaussiane, va molto meglio!

One step further: full KDE approximation! (RIASCOLTA QUESTA PARTE)

### K-Nearest Neighbor Classifier

Ho visto come si fa a classificare in base ai punti più vicini. Posso farlo anche qui! Ovviamente devi fare pre-processing, e specificare il tipo di distanza tra punti che devi calcolare.

Risultato dal notebook: THERE IS SOME OVERFITTING. k=1 è terribile.

Come decido qual è il K migliore? -> Cross-Validation! Risultato dal notebook: **8** è il valore migliore. (Nota che ha SCELTO di massimizzare l'accuracy = pochi false positives!)